# Class 14

[Twython Documentation](http://twython.readthedocs.io/en/latest/api.html)

In [1]:
api_key = "XXXX"
api_secret = "XXXX"
access_token = "XXXX"
token_secret = "XXXX"

In [2]:
import twython

In [ ]:
twitter = twython.Twython(api_key, api_secret, access_token, token_secret)

## Posting to Twitter 

In [ ]:
twitter.update_status(status="")

**Our workflow so far:**
+ [We created an app](http://apps.twitter.com/)
+ Authorized yourself to use that app using the "create token" button

**Getting access tokens for someone else:**
+ Create the app
+ Have the other user authorize the application

In order to make a Twitter bot, we need to replicate this process! 

Every twitter account needs to have a unique mobile phone number in order to make apps (which is why it is somewhat easier to start the application in your own account and then authorize another user than just start another account for your bot and start a new application there). 

Twitter attempts to stop you from making multiple accounts by tracking the cookies in your browser. We circumvent this by using a private window in Safari or an incognito window in Chrome. 

https://twitter.com/signup (skip the phone input) to create a new account for your Twitter bot. (Remember, we won't be able to just start a new application in that account because we won't have a phone number to associate with it). 

## Authorizing an app

Copy the URL from the output -- go to the window where you are logged in as your bot account, and THEN visit the URL. 

In [4]:
# Application mode 
twitter = twython.Twython(api_key, api_secret)
auth = twitter.get_authentication_tokens()
print("Log into Twitter as the user you want to authorize and visit this URL:")
print("\t" + auth['auth_url'])

Log into Twitter as the user you want to authorize and visit this URL:
	https://api.twitter.com/oauth/authenticate?oauth_token=6AnX5wAAAAAAv9B5AAABVcXQwtI


In [ ]:
# We copy the pin number provided when we visit the authorization URL 
pin = "XXXX"

twitter = twython.Twython(api_key, api_secret, auth['oauth_token'], auth['oauth_token_secret'])
tokens = twitter.get_authorized_tokens(pin)

new_access_token = tokens['oauth_token']
new_token_secret = tokens['oauth_token_secret']
print("your access token:", new_access_token)
print("your token secret:", new_token_secret)

# these tokens last forever! we can write a new application to post using this twitter bot. 
# But, if we wanted to post to a different account, we would need to redo this process. 

Now we're making API requests on behalf of the new bot account (rather than our own personal account)! 

In [6]:
twitter = twython.Twython(api_key, api_secret, new_access_token, new_token_secret)

In [8]:
twitter.update_status(status="hello!")

{'contributors': None,
 'coordinates': None,
 'created_at': 'Thu Jul 07 14:50:47 +0000 2016',
 'entities': {'hashtags': [], 'symbols': [], 'urls': [], 'user_mentions': []},
 'favorite_count': 0,
 'favorited': False,
 'geo': None,
 'id': 751065934643785728,
 'id_str': '751065934643785728',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'en',
 'place': None,
 'retweet_count': 0,
 'retweeted': False,
 'source': '<a href="http://www.columbia.edu/" rel="nofollow">ledebot</a>',
 'text': 'hello!',
 'truncated': False,
 'user': {'contributors_enabled': False,
  'created_at': 'Thu Jul 07 14:34:29 +0000 2016',
  'default_profile': True,
  'default_profile_image': True,
  'description': '',
  'entities': {'description': {'urls': []}},
  'favourites_count': 0,
  'follow_request_sent': False,
  'followers_count': 0,
  'following': False,
  'frien

## Implementing the bot

Eventually, we'd want to run out bot as a standalone python script (which can be run as a cronjob). 

**SAMPLE WORKFLOW**

Brainstorm and sketch in Jupyter notebook.

Create a standalone pythong scrip: bot.py

+ get data
+ modify data
+ post to twitter

Implement a cronjob to run script --> run every 3 hours

**TODAY, we want to:**

+ get a random lake from MONDIAL database
+ compose a sentence based on that lake's data
+ post the sentence to twitter

In [10]:
import pg8000
lakes = []

conn = pg8000.connect(database="mondial")
cursor = conn.cursor()

cursor.execute("SELECT name, area, depth, elevation, type, river from lake")
for row in cursor.fetchall():
    lake = {'name': row[0],
           'area': int(row[1]),
           'depth': int(row[2]),
           'elevation': int(row[3]),
           'type': row[4], 
           'river': row[5]}
    lakes.append(lake)

len(lakes)

143

In [ ]:
# Saving your data as json so you don't have to rely on your SQL database
import json
json.dump(lakes, open("lakes.json", "w"))

### Generate a fact sentence

* pick a column from the record 
* compose a sentence based on that column
* from a pre-existing list of templates

Templates: 
* `"The area of {} is {} square kilometers."`
* `"The depth of {} is {} meters."`
* etc.

Process:
* randomly select column
* if that column is not empty:
* ... fill in the template with the lake name and the value for that column. 

In [12]:
sentences = { 'area': 'The area of {} is {} square kilometers.',
            'depth': 'The depth of {} is {} meters.',
            'elevation': 'The elevation of {} is {} meters.',
            'type': 'The type of {} is {}.',
            'river': '{} empties into a river named {}.',}

In [13]:
import random

In [18]:
lake = random.choice(lakes)
col = random.choice(list(lake.keys()))

print(lake)
print(col)

sentence_template = sentences[col]
output = sentence_template.format(lake['name'], lake[col])
output

{'depth': Decimal('4'), 'river': 'Luapula', 'name': 'Lake Bangweulu', 'elevation': Decimal('1140'), 'area': Decimal('10000'), 'type': None}
depth


'The depth of Lake Bangweulu is 4 meters.'

In [19]:
# Let's make it a function! 

def random_lake_sentence(lakes, sentences):
    
    lake = random.choice(lakes)
    col = random.choice(list(lake.keys()))

    sentence_template = sentences[col]
    output = sentence_template.format(lake['name'], lake[col])
    return output

In [21]:
for i in range(10):
    print(random_lake_sentence(lakes, sentences))
    
# key error: name!
# we don't have a sentence template for name

The depth of Saimaa is 85 meters.


KeyError: 'name'

In [22]:
def random_lake_sentence(lakes, sentences):
    
    lake = random.choice(lakes)
    
    possible_keys = [k for k in lake.keys() if k != 'name']
    col = random.choice(possible_keys)

    sentence_template = sentences[col]
    output = sentence_template.format(lake['name'], lake[col])
    return output

In [23]:
for i in range(100):
    print(random_lake_sentence(lakes, sentences))

The depth of Lake Rukwa is 5 meters.
The type of Etoscha Salt Pan is salt.
The type of Lake Toba is caldera.
The area of Nam Co is 1855 square kilometers.
The depth of Lake Turkana is 73 meters.
Laguna Mar Chiquita empties into a river named None.
Lago di Bracciano empties into a river named None.
The elevation of Lago de Sobradinho is None meters.
The depth of Lago di Bolsena is 151 meters.
Chiemsee empties into a river named Alz.
The type of Lake Winnipeg is None.
Ozero Pskovskoje empties into a river named Narva.
The depth of Lago Trasimeno is 7 meters.
The depth of Lago di Bracciano is 160 meters.
The type of Kallavesi is None.
The depth of Lake Ohrid is 289 meters.
The type of Lake Tahoe is None.
The area of Vaettern is 1900 square kilometers.
The area of Lake Maracaibo is 13000 square kilometers.
The area of Brienzersee is 29.8 square kilometers.
The elevation of Kremenchuk Reservoir is 44 meters.
The depth of Lac Assal is None meters.
The type of Lake Maracaibo is None.
The type

In [24]:
def random_lake_sentence(lakes, sentences):
    
    lake = random.choice(lakes)
    
    possible_keys = [k for k in lake.keys() if k != 'name' and lake[k] is not None]
    col = random.choice(possible_keys)

    sentence_template = sentences[col]
    output = sentence_template.format(lake['name'], lake[col])
    return output

In [25]:
for i in range(100):
    print(random_lake_sentence(lakes, sentences))

The elevation of Lake Turkana is 375 meters.
The type of Kiev Reservoir is dam.
The depth of Koli Sarez is 500 meters.
The depth of Laguna de Bay is 20 meters.
The elevation of Ozero Tschany is 10 meters.
The area of Lake Burley Griffin is 6.64 square kilometers.
The elevation of Lago Trasimeno is 259 meters.
The elevation of Lake Bosumtwi is 107 meters.
The area of Lake Burley Griffin is 6.64 square kilometers.
The elevation of Lake Turkana is 375 meters.
The depth of Zurichsee is 136 meters.
The elevation of Ozero Tschany is 10 meters.
The area of Lago di Como is 146 square kilometers.
The depth of Malebo Pool is 10 meters.
Lake Winnipesaukee empties into a river named Merrimack River.
The area of Lake Champlain is 1130 square kilometers.
The area of Lake Kioga is 1720 square kilometers.
The depth of Lake Huron is 229 meters.
The depth of Thunersee is 217 meters.
The area of Ozero Onega is 9616 square kilometers.
The depth of Lake Sakakawea is 55 meters.
The area of Lake Bangweulu is

In [28]:
flare = ['Wow!', "Cool, huh?", 'Now you know.', 'WHAAAAT', 'Neat-o.']

output = random_lake_sentence(lakes, sentences) + " " + random.choice(flare)
twitter.update_status(status=output)

{'contributors': None,
 'coordinates': None,
 'created_at': 'Thu Jul 07 15:35:39 +0000 2016',
 'entities': {'hashtags': [], 'symbols': [], 'urls': [], 'user_mentions': []},
 'favorite_count': 0,
 'favorited': False,
 'geo': None,
 'id': 751077226561634304,
 'id_str': '751077226561634304',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'en',
 'place': None,
 'retweet_count': 0,
 'retweeted': False,
 'source': '<a href="http://www.columbia.edu/" rel="nofollow">ledebot</a>',
 'text': 'The depth of Lake Okeechobee is 4 meters. Now you know.',
 'truncated': False,
 'user': {'contributors_enabled': False,
  'created_at': 'Thu Jul 07 14:34:29 +0000 2016',
  'default_profile': True,
  'default_profile_image': True,
  'description': '',
  'entities': {'description': {'urls': []}},
  'favourites_count': 0,
  'follow_request_sent': False,
  'fo

In your standalone python script, remember that you'd want to copy over your bot's access token and secret (rather than your own). 